### Graded Lab 4

Hello ! Welcome to Graded Lab of Module 4.

In the last assignment we had worked on correlation , hypothesis which business wanted us to test.
Its high time when we look at the missing values & outliers stuff.
**Here we will work on missing values & outliers.**

In case if you are not able to recollect the problem description and data description then mentioning it below.

Lets look at the problem statement,

*Client: ABC Retail, Incorporated, rest-of-the-world division* 

***Project name: Online retail sales analysis*** 

An online retailer, ABC, Inc., operates in nearly 100 countries worldwide, selling furniture, office supplies and technology products to customers in three segments: consumer, corporate and home office. ABC, Inc. is a US-based company, and it has two major divisions: US and rest of the world. We are working with the rest of the world division of the company. 

They have provided us with online sales transaction data from 2011 to 2014.

We are given 3 datasets:-

1. Data on each sale; 51290 records; all data in US dollars
It contains fields like
**order_id** (identifier) ,order_date ,ship_date ,ship_mode ,**customer_id**(identifier) ,product_id ,category ,sub_category ,product_name ,sales ,quantity ,discount ,profit ,shipping_cost ,order_priority ,**vendor_code** (identifier) 


2. Data on the customers; 1590 records 
It contains fields like
**customer_id** (identifier) ,customer_name ,city ,state ,country ,postal_code ,segment ,market ,region 

3. Data on vendors who supply the retailer; 65 records 
It contains fields like
vendor ,**vendor_code** (identifier) 

We need to analyze the data and need to provide answer to different questions asked by company officials.

In [130]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from itertools import combinations
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

### Reading sales data
sales = pd.read_csv('sales_data_M4.csv')

### Reading customer data
cust = pd.read_csv(r'customers.csv',encoding='iso-8859-1')

### Reading vendor data
vend = pd.read_csv(r'vendors.csv')

sales.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,vendor_code
0,1,MX-2014-143658,02/10/2014,06/10/2014,Standard Class,SC-20575,OFF-LA-10002782,Office Supplies,Labels,"Hon File Folder Labels, Adjustable",13.08,3,0.0,4.56,1.03,Medium,VE_001
1,2,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,FUR-FU-10004015,Furniture,Furnishings,"Tenex Clock, Durable",252.16,8,0.0,90.72,13.45,Medium,VE_002
2,3,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,FUR-BO-10002352,Furniture,Bookcases,"Ikea 3-Shelf Cabinet, Mobile",193.28,2,0.0,54.08,9.63,Medium,VE_003
3,4,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,OFF-BI-10004428,Office Supplies,Binders,"Cardinal Binder, Clear",35.44,4,0.0,4.96,1.37,Medium,VE_004
4,5,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,OFF-AR-10004594,Office Supplies,Art,"Sanford Canvas, Water Color",71.60,2,0.0,11.44,3.79,Medium,VE_005


In order to solve the next questions , we need to combine all the 3 datasets into a single dataframe such that every details of sales dataframe are intact. So here we have written a data processing function.
There are 2 tasks which are to be performed.
1. Merge/ Join all the 3 datasets into a single dataframe such that every details of sales dataframe are intact. (Understand which should be the joining key , type of join , refer .merge() function of pandas)
2. Convert 'order_date' into a datetime column.
**Return output as a dataframe**

In [131]:
#### data_merging & order_date processing , data1 will be sales , data2 will be customer dataset & data3 will be vendor dataset.

def data_process(data1,data2,data3):
    # your code here
    data = pd.merge(data1,data2,on='customer_id',how='left')
    data = pd.merge(data,data3,on='vendor_code',how='left')
    data['order_date'] = pd.to_datetime(data['order_date'])
    return data

In [132]:
sales= data_process(data1=sales.copy(),data2=cust.copy(),data3=vend.copy())

In [133]:
sales.sample(100).head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,product_id,category,sub_category,product_name,...,vendor_code,customer_name,city,state,country,postal_code,segment,market,region,vendor
16683,16684,ES-2012-5959969,2012-10-30,04/11/2012,Standard Class,AJ-10780,OFF-AP-10004869,Office Supplies,Appliances,"KitchenAid Toaster, White",...,VE_018,Anthony Jacobs,Chihuahua,Chihuahua,Mexico,NaN,Corporate,LATAM,LATAM-North,Proton Solutions
29127,29128,ID-2012-22550,2012-11-13,17/11/2012,Standard Class,BD-11620,OFF-ST-10003931,Office Supplies,Storage,"Smead Lockers, Blue",...,VE_037,Brian DeCherney,Santiago,Santiago,Chile,NaN,Consumer,LATAM,LATAM-South,Riverlife
20517,20518,ID-2014-53679,2014-10-29,05/11/2014,Standard Class,PP-18955,TEC-PH-10004618,Technology,Phones,"Motorola Speaker Phone, VoIP",...,VE_038,Paul Prost,Santo Domingo,Santo Domingo,Dominican Republic,NaN,Home Office,LATAM,Caribbean,Spicurity
48261,48262,MZ-2014-4670,2014-06-24,29/06/2014,Standard Class,AZ-750,OFF-IBI-10004855,Office Supplies,Binders,"Ibico Hole Reinforcements, Recycled",...,VE_060,Annie Zypern,Uskudar,Istanbul,Turkey,NaN,Consumer,EMEA,EMEA,Drift Co.
47415,47416,BO-2014-170,2014-06-25,01/07/2014,Standard Class,TC-11145,FUR-TEN-10003349,Furniture,Furnishings,"Tenex Photo Frame, Durable",...,VE_031,Theresa Coyne,Iskenderun,Hatay,Turkey,NaN,Corporate,EMEA,EMEA,Sail Furniture


In [134]:
assert sales['order_date'].dtypes=='<M8[ns]' ,'Make sure if you have converted order_date into a datetime format correctly or not.'
assert sales.shape== (51290,26) ,'Checking size and shape of dataframe after merging is a very important check.'

Note that in this dataset we are purposefully introducing some of the missing values.

### Q.1 Return column names & missing values percentage for the columns which has missing value% >50. Output should be a dictionary. 

For eg:- { column_name : value%} i.e {'A':54.50,'B':59.75}, make sure to round off percentages to 2 decimals.

In [135]:
def missing_value_col(data):
    # your code here
    missing_data = data.isnull().sum()/len(data)*100
    missing_data = missing_data.to_dict()
    missing_cols = {key:missing_data[key] for key in missing_data.keys() if missing_data[key]>50}
    return missing_cols

In [136]:
missing_value_col(data=sales)

{'postal_code': 100.0}

In [137]:
list(missing_value_col(data=sales).keys())

['postal_code']

In [138]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.

#### **Senior data scientists after consulting with the business have decided that such columns needs to be dropped before going further.**

In [139]:
high_missing_col=list(missing_value_col(data=sales).keys()) #Enter the columns with high % of missing values
### We will drop columns with such high missing values
sales.drop(high_missing_col,axis=1,inplace=True)

In [140]:
sales.columns

Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode',
       'customer_id', 'product_id', 'category', 'sub_category', 'product_name',
       'sales', 'quantity', 'discount', 'profit', 'shipping_cost',
       'order_priority', 'vendor_code', 'customer_name', 'city', 'state',
       'country', 'segment', 'market', 'region', 'vendor'],
      dtype='object')

### Q.2 Fill the missing data in 'category' column with appropriate method.  
(Hint:- Remember variable 'category' & 'sub-category' are related & hierachy goes like this category -> sub-category)

Approach:-

1. Create a key-pair dictionary where key will be product category & pair values will be sub-categories.

2. Wherever category value is missing check for corresponding sub-category value. 

3. From sub-category value trace back its original product category value by looking in dictionary, get a product value.

4. Fill the blank product category value by the same.

In [141]:
sales.sample(100).head(2)

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,product_id,category,sub_category,product_name,...,order_priority,vendor_code,customer_name,city,state,country,segment,market,region,vendor
9578,9579,MX-2011-155257,2011-07-13,15/07/2011,First Class,JH-16180,OFF-ST-10001172,NaN,Storage,"Fellowes Trays, Wire Frame",...,High,VE_006,Justin Hirsh,Mexico City,Distrito Federal,Mexico,Consumer,LATAM,LATAM-North,Venusystems
21122,21123,IN-2012-74686,2012-12-17,21/12/2012,Standard Class,JH-15910,OFF-LA-10000471,Office Supplies,Labels,"Smead Shipping Labels, 5000 Label Set",...,High,VE_030,Jonathan Howell,Matagalpa,Matagalpa,Nicaragua,Consumer,LATAM,LATAM-Central,Cubemaster


In [142]:
### create a dictionary where keys will be categories & sub-categories as values. store in inside Category_dict.

def cat_dict(data,main_col,sub_col):    
    # your code here
    categories = data[[main_col,sub_col]].drop_duplicates()
    Category_dict = categories.groupby(main_col)[sub_col].apply(list).to_dict()
    return Category_dict

def category_subcategory_map_dataframe(df, Category_dict, main_col, sub_col):
    def category_subcategory_map(row):
        if pd.isna(row[main_col]):  # If the main column is null
            if not pd.isna(row[sub_col]):  # If the sub column is not null
                for k, v in Category_dict.items():
                    if row[sub_col] in Category_dict[k]:
                        return k  # k is the correct sub-category for the category value v
        else:
            return row[main_col]  # If the main column is not null, return its existing value

    df[main_col] = df.apply(category_subcategory_map, axis=1)
    return df[main_col]

In [143]:
cat_dict(data=sales,main_col='category',sub_col='sub_category')

{'Furniture': ['Furnishings', 'Bookcases', 'Chairs', 'Tables'],
 'Office Supplies': ['Labels',
  'Binders',
  'Art',
  'Envelopes',
  'Supplies',
  'Fasteners',
  'Storage',
  'Appliances',
  'Paper'],
 'Technology': ['Machines', 'Accessories', 'Copiers', 'Phones']}

In [144]:
category_dict=cat_dict(data=sales,main_col='category',sub_col='sub_category')
category_subcategory_map_dataframe(df=sales, Category_dict=category_dict, main_col='category', sub_col='sub_category')

0        Office Supplies
1              Furniture
2              Furniture
3        Office Supplies
4        Office Supplies
              ...       
51285    Office Supplies
51286         Technology
51287    Office Supplies
51288    Office Supplies
51289    Office Supplies
Name: category, Length: 51290, dtype: object

In [145]:
Category_dict=cat_dict(data=sales,main_col='category',sub_col='sub_category')
sales['category']=category_subcategory_map_dataframe(df=sales, Category_dict=Category_dict, main_col='category', sub_col='sub_category')

In [146]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.

### Q.3 ) Fill the missing values in 'sales' column by average value of that column. Return output series.

In [147]:
def fill_missing(data,col):
    # your code here
    data[col].fillna(data[col].mean(),inplace=True)
    return data[col]

In [148]:
sales.isna().sum()

row_id               0
order_id             0
order_date           0
ship_date            0
ship_mode         1570
customer_id          0
product_id           0
category             0
sub_category         0
product_name      1297
sales             6204
quantity             0
discount             0
profit               0
shipping_cost        0
order_priority    1046
vendor_code          0
customer_name        0
city                 0
state                0
country              0
segment              0
market               0
region               0
vendor               0
dtype: int64

In [149]:
fill_missing(data=sales,col='sales').isna().sum()

0

In [150]:
sales['sales']=fill_missing(data=sales,col='sales')

In [151]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert sales['sales'].isna().sum()==0 ,'Missing values found for sales.'

### Q.4 ) Discretize the 'sales' column wrto following groups. Each group should contain number of orders as values.
Groups :- [0,12),[12,37),[37,113),[113,186),[186,1200).

**Note :-[a,b) indicates that group includes a but excudes b.**

**Return output series with group name as index & number of orders as values.**

For eg:-

[186, 1200)             123

[37, 113)               123

[12, 37)                123

[113, 186)              123

[0, 12)                 123

Name: sales, dtype: int64

In [152]:
sales.sample(100).head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,product_id,category,sub_category,product_name,...,order_priority,vendor_code,customer_name,city,state,country,segment,market,region,vendor
32176,32177,CA-2014-111689,2014-01-12,03/12/2014,Second Class,HP-14815,FUR-CH-10004287,Furniture,Chairs,SAFCO Arco Folding Chair,...,Critical,VE_056,Harold Pawlan,Monterrey,Nuevo León,Mexico,Home Office,LATAM,LATAM-North,Purpleshack
32286,32287,CA-2012-153549,2012-03-29,31/03/2012,Second Class,SL-20155,FUR-CH-10004086,Furniture,Chairs,Hon 4070 Series Pagoda Armless Upholstered Sta...,...,Medium,VE_056,Sara Luxemburg,Ilopango,San Salvador,El Salvador,Home Office,LATAM,LATAM-Central,Purpleshack
31647,31648,CA-2013-129714,2013-02-09,04/09/2013,First Class,AB-10060,OFF-BI-10002160,Office Supplies,Binders,Acco Hanging Data Binders,...,Critical,VE_016,Adam Bellavance,Soyapango,San Salvador,El Salvador,Home Office,LATAM,LATAM-Central,Lifoods
15641,15642,ES-2011-2535754,2011-08-03,13/03/2011,Standard Class,LW-17215,OFF-FA-10000670,Office Supplies,Fasteners,"Advantus Clamps, Assorted Sizes",...,Medium,VE_018,Luke Weiss,Villa Nueva,Guatemala,Guatemala,Consumer,LATAM,LATAM-Central,Proton Solutions
50117,50118,AG-2011-8180,2011-02-25,25/02/2011,Same Day,RO-9780,TEC-EPS-10004558,Technology,Machines,"Epson Inkjet, Durable",...,Critical,VE_022,Rose O'Brian,Al Mahallah al Kubra,Al Gharbiyah,Egypt,Consumer,Africa,Africa,Mermaid Industries


In [153]:
def manual_bin(data,col):
    # your code here
    bin_edges = [0,12,37,113,186,1200]
    data[col+'_bin'] = pd.cut(data[col],\
                             bins = bin_edges,\
                              right = False,
                             include_lowest = True)
    return data[col+'_bin'].value_counts()

In [154]:
manual_bin(data=sales,col='sales')

[186, 1200)    18550
[37, 113)      12761
[12, 37)        9325
[113, 186)      5187
[0, 12)         3708
Name: sales_bin, dtype: int64

In [155]:
assert type(manual_bin(data=sales,col='sales'))==pd.Series, "Please provide output in series format."

In [156]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert manual_bin(data=sales,col='sales').values[0]==18550, 'Make sure you have returned correct series values.'

### Q.5 )  Sometimes, it is difficult to comprehend the quantity sold ('quantity') value as a numeric value. So, the business team wants to observe the sales as High, Medium and Low categories.  

### Write a code to discretize the sales data and  return an output series containing High, Medium and Low categories as index & the respective number of records as values.

Hint : Here the bounds for High , Medium , Low categories are not given hence we can consider the equal split of data for defining the bounds , think of using pd.cut() function.

In [157]:
# turning the numeric data in discretized data
data = pd.cut(sales['quantity'], bins = 3,include_lowest=True)#,labels = ['Low', 'Medium', 'High'])
data.value_counts()

(0.986, 5.333]    42660
(5.333, 9.667]     7753
(9.667, 14.0]       877
Name: quantity, dtype: int64

In [158]:
data = pd.cut(sales['quantity'], bins = 3,include_lowest=True,labels = ['Low', 'Medium', 'High'])
data.value_counts()

Low       42660
Medium     7753
High        877
Name: quantity, dtype: int64

In [159]:
def discrete_bin(data,col):
    # your code here
    data_bin = pd.cut(data[col], bins = 3,include_lowest=True,labels = ['Low', 'Medium', 'High'])
    return data_bin.value_counts()

In [160]:
assert type(discrete_bin(data=sales,col='quantity'))==pd.Series, "Please provide output in series format."

In [161]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.

### Q.6. In order to perform further analysis, we intend to utilize the 'ship_mode' variable, which has categorical values ('First Class'(0) > 'Second Class'(1) > 'Standard Class'(2) > 'Same Day'(3)). To facilitate this analysis, we need to convert 'ship_mode' into a numerical feature. Please encode 'ship_mode' into a numerical format.
### Return output series.

Hint : Think of using the OrdinalEncoder() function from scikit-learn.

In [162]:
sales[['ship_mode','order_priority']].head()

,ship_mode,order_priority
0,Standard Class,Medium
1,Standard Class,Medium
2,Standard Class,Medium
3,Standard Class,Medium
4,Standard Class,Medium


In [163]:
sales['order_priority'].unique()

array(['Medium', 'Low', 'High', nan, 'Critical'], dtype=object)

In [164]:
sales[['ship_mode']]

,ship_mode
0,Standard Class
1,Standard Class
2,Standard Class
3,Standard Class
4,Standard Class
...,...
51285,First Class
51286,NaN
51287,First Class
51288,First Class


In [165]:
priority = ['First Class','Second Class','Standard Class','Same Day','nan_val']
nan_val=5
from sklearn.preprocessing import OrdinalEncoder
import numpy as np
encoder = OrdinalEncoder(categories=[priority])
data = sales.copy()
col = 'ship_mode'
data['ship_mode'].fillna('nan_val',inplace=True)
enc_shipmode = encoder.fit_transform(data[['ship_mode']])
enc_shipmode = np.where(enc_shipmode==4,nan_val,enc_shipmode)
enc_shipmode[-5:]

array([[0.],
       [5.],
       [0.],
       [0.],
       [2.]])

In [166]:
## Select Appropriate priority
priority = ['First Class','Second Class','Standard Class','Same Day','nan_val'] # Write your code in place of None 
def transforming_ordered_var(data,col,priority,nan_val):
    #data: dataset to be used
    #col : column name which is to be encoded.
    #priority: list of categories sorted from highest order of importance to lowest order importance.
    #nan_val : it will be a value assigned in place of nan's. 
    #
    # your code here
    from sklearn.preprocessing import OrdinalEncoder
    import numpy as np
    import pandas as pd
    encoder = OrdinalEncoder(categories=[priority])
#     data = sales.copy()
#     col = 'ship_mode'
    data['ship_mode'].fillna('nan_val',inplace=True)
    enc_shipmode = encoder.fit_transform(data[['ship_mode']])
    enc_shipmode = enc_shipmode.reshape(-1)
    enc_shipmode = np.where(enc_shipmode==4,nan_val,enc_shipmode)
    enc_shipmode = pd.Series(enc_shipmode)
    return enc_shipmode

In [167]:
sales['shipping_mode_encoded']=transforming_ordered_var(data=sales,col='ship_mode',priority=priority,nan_val=5)

In [168]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.
assert int(transforming_ordered_var(data=sales,col='ship_mode',priority=priority,nan_val=5).value_counts().loc[lambda x:x.index==5])==1570

### Q.7) Outliers can be defined in different ways. For each of the following definitions of outliers, what percent of the values in the  'sales' are considered outliers? Give the answer as a percent rounded upto two decimal places.


Note -
1. 'IQR_detection' : Less than (Q1 - 1.5*IQR) and greater than (Q3 + 1.5*IQR) are considered outliers

2. 'percentile_detection': Below the 3rd percentile and above the 97th percentile are considered outliers

3. 'mean_SD' : Less than (mean - 3*SD) and greater than (mean + 3*SD) are considered outliers 

In [169]:
data = sales.copy()
col = 'sales'
Q1 = data[col].quantile(0.25)
Q3 = data[col].quantile(0.75)
IQR = Q3 - Q1
outliers = data[(data[col] < (Q1 - 1.5 * IQR)) | (data[col] > (Q3 + 1.5 * IQR))]
outliers_perc = round(len(outliers)/len(sales),2)

In [170]:
data = sales.copy()
col = 'sales'
outliers = data[(data[col] < data[col].quantile(0.03)) | (data[col] > data[col].quantile(0.97))]
outliers_perc = round(len(outliers)*100/len(sales),2)

In [171]:
data = sales.copy()
col = 'sales'
mean = data[col].mean()
SD = data[col].std()
outliers = data[(data[col] < (mean - 3*SD)) | (data[col] > (mean + 3*SD))]
outliers_perc = round(len(outliers)*100/len(sales),2)

In [172]:
def detect_outliers(data,col,method):
    if method=='IQR_detection':
        # your code here
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = data[(data[col] < (Q1 - 1.5 * IQR)) | (data[col] > (Q3 + 1.5 * IQR))]
        outliers_perc = round(len(outliers)*100/len(sales),2)
    elif method=='percentile_detection':
        # your code here
        outliers = data[(data[col] < data[col].quantile(0.03)) | (data[col] > data[col].quantile(0.97))]
        outliers_perc = round(len(outliers)*100/len(sales),2)
    elif method=='mean_SD':
        # your code here
        mean = data[col].mean()
        SD = data[col].std()
        outliers = data[(data[col] < (mean - 3*SD)) | (data[col] > (mean + 3*SD))]
        outliers_perc = round(len(outliers)*100/len(sales),2)
    return outliers_perc

In [173]:
assert type(detect_outliers(data=sales,col='sales',method='IQR_detection'))==np.float, 'Make sure that output is returned as float'
assert detect_outliers(data=sales,col='sales',method='IQR_detection')==10.06, 'Make sure that output is rounded upto 2 decimals'

In [174]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.

### Q.8) Detect upper end outliers in profit column by 'IQR_detection'. Return number of upper end outliers.

In [175]:
data = sales.copy()
col = 'profit'
Q1 = data[col].quantile(0.25)
Q3 = data[col].quantile(0.75)
IQR = Q3 - Q1
upper_outliers = data[data[col] > (Q3 + 1.5 * IQR)]
upper_outliers.shape[0]

6229

In [176]:
def detect_upperend_outliers(data,col):
    # your code here
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = data[data[col] > (Q3 + 1.5 * IQR)]
    outliers.shape
    return outliers.shape[0] ##data frame named 'outliers' which contains only outliers.

In [177]:
assert type(detect_upperend_outliers(data=sales,col='profit'))==int , 'No of outliers should be an integer , make sure that you are returning output in integer format.'

In [178]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.

### Q.9) Cap & Floor outliers at 97% at 3%. Return the processed column as output.

In [179]:
data = sales.copy()
col = 'profit'
lower_limit = data[col].quantile(0.03)
upper_limit = data[col].quantile(0.97)
capped_col = data[col]
print(lower_limit,upper_limit)
capped_col = np.where(capped_col<=lower_limit,lower_limit,capped_col)
capped_col = np.where(capped_col>=upper_limit,upper_limit,capped_col)
print(np.quantile(capped_col,0.03),np.quantile(capped_col,0.97))

-144.63594 301.97196
-144.6319602 301.9693068


In [180]:
def cap_and_floor_column(column, cap_percentile, floor_percentile):
    # your code here
    lower_limit = data[col].quantile(floor_percentile/100)
    upper_limit = data[col].quantile(cap_percentile/100)
    capped_col = data[col]
    capped_col = np.where(capped_col<=lower_limit,lower_limit,capped_col)
    capped_col = np.where(capped_col>=upper_limit,upper_limit,capped_col)
    return capped_col ## Capped ,Floored series

In [181]:
sales['profit_outlier_capped'] = cap_and_floor_column(column=sales['profit'], cap_percentile=97, floor_percentile=3)

In [182]:
assert round(np.quantile(sales['profit_outlier_capped'],0.97),4)==301.9693, 'Make sure that you are flooring and capping with correct percentiles & returning the correct output.'
assert round(np.quantile(sales['profit_outlier_capped'],0.03),4)==-144.632, 'Make sure that you are flooring and capping with correct percentiles & returning the correct output.'

In [183]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.

### Q.10) Business team is interested in knowing how many vendors have profit_outlier_capped <10000 & vendors with profit_outlier_capped >10000. Return the dataframe with 3 columns vendor  , profit_outlier_capped , profit_bins & sort it with 'profit_outlier_capped' in descending order.

Instructions:-
1. Bins should be in (a,b] format. (a,b] indicates that bin excludes a but incudes b.
2. Make sure that for **1st interval only** it should be **left inclusive** i.e [a,b].
3. Segment the ''profit_outlier_capped'' variable in 2 bins. (binendpoints should be min value of profit<10000,10000,max value of profit>10000).

In [199]:
data = sales.copy()
col = 'profit_outlier_capped'
maxx = data['profit_outlier_capped'].max()
minn = data['profit_outlier_capped'].min()
print(minn,maxx)
bin_edges = [minn,100,maxx]
data['profit_bins'] = pd.cut(data[col],\
                         bins = bin_edges,\
                          right = True,
                         include_lowest = True)

data1 = data[['vendor','profit_outlier_capped','profit_bins']]
data1 = data1.sort_values(by=['profit_outlier_capped'],axis=0,ascending=False)
data1.head()

-144.63594 301.97196


,vendor,profit_outlier_capped,profit_bins
24861,Gnomestone,301.97196,"(100.0, 301.972]"
20520,Venusystems,301.97196,"(100.0, 301.972]"
20686,Sail Furniture,301.97196,"(100.0, 301.972]"
7649,Nero,301.97196,"(100.0, 301.972]"
7653,Energy Systems,301.97196,"(100.0, 301.972]"


In [200]:
def profit_vendor(data,col):
    # your code here
    maxx = data['profit_outlier_capped'].max()
    minn = data['profit_outlier_capped'].min()
    print(minn,maxx)
    bin_edges = [minn,100,maxx]
    data['profit_bins'] = pd.cut(data[col],\
                             bins = bin_edges,\
                              right = True,
                             include_lowest = True)

    data = data[['vendor','profit_outlier_capped','profit_bins']]
    data = data.sort_values(by=['profit_outlier_capped'],axis=0,ascending=False)
    return data

In [201]:
assert type(profit_vendor(data=sales,col='profit_outlier_capped'))==pd.DataFrame ,'Make sure that your output is a dataframe '
assert str(profit_vendor(data=sales,col='profit_outlier_capped')['profit_bins'].unique()[0])=='(10000.0, 76469.853]' , 'Make sure that you are selecting bin cut-points(a & b) properly'
assert str(profit_vendor(data=sales,col='profit_outlier_capped')['profit_bins'].unique()[1])=='(2732.9049999999997, 10000.0]', 'Make sure that you are selecting bin cut-points(a & b) properly'

-144.63594 301.97196
-144.63594 301.97196


AssertionError: Make sure that you are selecting bin cut-points(a & b) properly

In [ ]:
# autograder cells , please do not alter/ delete /edit this cell,Kindly ignore this cell.